# PQ and Faiss

In this notebook we will cover the product quantization (PQ) implementation in Faiss. First, we need to load our Sift1M dataset.

In [1]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

# 1M samples, cut down to 500K
xb = read_fvecs('../../../data/sift/sift_base.fvecs')[:500_000]
# queries
xq = read_fvecs('../../../data/sift/sift_query.fvecs')[0].reshape(1, -1)

Initialize an index in Faiss using *only* PQ.

In [2]:
import faiss

D = xb.shape[1]
m = 8
assert D % m == 0
nbits = 8  # number of bits per subquantizer, k* = 2**nbits
index = faiss.IndexPQ(D, m, nbits)

We have our cluster centroids (*reproduction values*) that must be optimized to our dataset, therefore if we check if the `index` `is_trained` we will see it is *not*.

In [3]:
index.is_trained

False

So we must train it using our vectors `xb`.

In [4]:
index.train(xb)  # PQ training can take some time when using large nbits

And now we can `add` our vectors and `search`.

In [5]:
index.add(xb)  # this is also very slow for large nbits

k = 100  # return top k results

In [6]:
dist, I = index.search(xq, k)

CPU times: user 22.7 ms, sys: 13.2 ms, total: 35.9 ms
Wall time: 9.37 ms


In [7]:
%%timeit
index.search(xq, k)

1.49 ms ± 49.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Search time is nothing special, PQ alone is still an exhaustive search so we would expect nothing spectacular here - but we can make it fast as we'll see later.

Let's compare our results against those produced by a non-quantized flat index.

In [8]:
l2_index = faiss.IndexFlatL2(D)
l2_index.add(xb)

In [9]:
%%time
l2_dist, l2_I = l2_index.search(xq, k)

CPU times: user 46.1 ms, sys: 15.1 ms, total: 61.2 ms
Wall time: 15 ms


In [10]:
sum([1 for i in I[0] if i in l2_I])

50

A recall of 50%, not cutting-edge, but a reasonable sacrifice if this allows us to search larger datasets. Let's see if PQ has made good on it's promise of reduced memory usage.

In [11]:
import os

def get_memory(index):
    # write index to file
    faiss.write_index(index, './temp.index')
    # get file size
    file_size = os.path.getsize('./temp.index')
    # delete saved index
    os.remove('./temp.index')
    return file_size

In [12]:
# first we check our Flat index, this should be larger
get_memory(l2_index)

256000045

In [13]:
# now our PQ index
get_memory(index)

4131158

We've reduced our index size from a cool **256MB**, to just **4.1MB** - that is pretty impressive.

But what about this not so great search-speed? Is there anything we can do about that? Fortunately yes! We can improve search-speed by using another quantization step - we add a *coarse* quantizer, `IndexIVF` to the process.

What we will now have is a *coarse quantizer* (IVF) which restricts the *scope of our search* to the most relevant samples - followed by our *fine quantizer* (PQ) which compresses the high-memory usage of our stored vectors by *restricting the scope of our vector**space***.

In [14]:
vecs = faiss.IndexFlatL2(D)

nlist = 2048  # how many Voronoi cells (must be >= k* which is 2**nbits)
nbits = 8  # when using IVF+PQ, higher nbits values are not supported
index = faiss.IndexIVFPQ(vecs, D, nlist, m, nbits)
print(f"{2**nbits=}")  # our value for nlist

2**nbits=256


In [15]:
index.is_trained

False

Again we must `train` our index, and again this can take awhile.

In [16]:
index.train(xb)

Now we `add` our vectors.

In [17]:
index.add(xb)

And now we're ready to test our new IVF -> PQ index!

In [18]:
dist, I = index.search(xq, k)

In [19]:
%%timeit
index.search(xq, k)

86.3 µs ± 15 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


*Lightning fast search...* But how is the recall?

In [20]:
sum([1 for i in I[0] if i in l2_I])

34

Our recall limit will be roughly what we reached when using `IndexPQ` alone, so we have hit 36/57, or about 60% of the potential recall performance. Now, because we are now using an IVF index, we can push our recall towards these higher values by increasing the number of cells to search using `nprobe`. Although it is worth noting that in this case, as there is always some noise in training the index, the max recall achievable is 43%.

In [21]:
index.nprobe = 2

In [22]:
dist, I = index.search(xq, k)

In [23]:
sum([1 for i in I[0] if i in l2_I])

39

In [24]:
index.nprobe = 48
dist, I = index.search(xq, k)
sum([1 for i in I[0] if i in l2_I])

52

52%, getting better... Now, if we were to try pushing the `nprobe` value up signficantly, we won't return any recall results better than 52%, this is our best-case performance in terms of balancing recall and search-time.

In [25]:
index.nprobe = 2048  # this is searching all Voronoi cells
dist, I = index.search(xq, k)
sum([1 for i in I[0] if i in l2_I])

52

So it is better that we stick with `nprobe` of `48`. Let's take a look at the speed difference between this index and a flat L2 index.

In [26]:
index.nprobe = 48

In [27]:
%%timeit
dist, I = index.search(xq, k)

134 µs ± 4.89 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [28]:
%%timeit
l2_dist, l2_I = l2_index.search(xq, k)

8.42 ms ± 344 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


A significant speed increase from 8.42ms to 132µs, and what are the differences in memory usage?

In [29]:
print(f"IVFPQ: {get_memory(index)}\nFlat: {get_memory(l2_index)}")

IVFPQ: 9196212
Flat: 256000045


So our final stats are:

| | FlatL2 | PQ | IVFPQ | Note |
| --- | --- | --- | --- | --- |
| Recall | 100% | 50% | 52% | *IVFPQ and PQ can attain equal performance with equivalent parameters - improved recall with greater `nbits`* |
| Speed | 8.26ms | 1.49ms | 0.29ms | |
| Memory | 256MB | 6.5MB | 9.2MB | |